In [1]:
import geopandas as gpd
import rasterio as rio
from rasterio.mask import mask

In [2]:
# ZIP file Sentinel-2
data_path = r'C:\SKOLA\02_DOKTORAT\ml_dpz\lcz\berlin_data\S2B_MSIL2A_20170519T101019_N0500_R022_T33UVT_20231115T232025.SAFE.zip'

# GT raster converted to polygon
polygon_path = r'C:\SKOLA\02_DOKTORAT\ml_dpz\lcz\berlin_data\berlin_polygon.shp'

print(data_path)
print(polygon_path)

image_name = data_path.split('\\')[-1]
image_name = image_name.split('.')[0]

C:\SKOLA\02_DOKTORAT\ml_dpz\lcz\berlin_data\S2B_MSIL2A_20170519T101019_N0500_R022_T33UVT_20231115T232025.SAFE.zip
C:\SKOLA\02_DOKTORAT\ml_dpz\lcz\berlin_data\berlin_polygon.shp


In [3]:
polygon = gpd.read_file(polygon_path)
polygon.explore()

In [4]:
polygon_buffered = polygon['geometry'].buffer(500)
polygon_buffered.explore()

In [5]:
with rio.open(data_path) as ds:
    sd = ds.subdatasets
    meta = ds.meta

print(len(sd))
print(sd[0])
print(sd[1])

4
SENTINEL2_L2A:/vsizip/C:/SKOLA/02_DOKTORAT/ml_dpz/lcz/berlin_data/S2B_MSIL2A_20170519T101019_N0500_R022_T33UVT_20231115T232025.SAFE.zip/S2B_MSIL2A_20170519T101019_N0500_R022_T33UVT_20231115T232025.SAFE/MTD_MSIL2A.xml:10m:EPSG_32633
SENTINEL2_L2A:/vsizip/C:/SKOLA/02_DOKTORAT/ml_dpz/lcz/berlin_data/S2B_MSIL2A_20170519T101019_N0500_R022_T33UVT_20231115T232025.SAFE.zip/S2B_MSIL2A_20170519T101019_N0500_R022_T33UVT_20231115T232025.SAFE/MTD_MSIL2A.xml:20m:EPSG_32633


C:\Users\Lenovo\AppData\Roaming\Python\Python39\site-packages\rasterio\__init__.py:333: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [6]:
# we want subdata 0 a 1
sds=[0,1]

In [7]:
with rio.open(sd[0]) as b10m:
    sentinel_crs = rio.crs.CRS.to_epsg(b10m.crs)
    print(f'Sentinel CRS: {sentinel_crs}')

Sentinel CRS: 32633


In [8]:
polygon_crs = polygon.crs.to_epsg()
print(f'Polygon CRS: {polygon_crs}')

Polygon CRS: 32633


In [9]:
if sentinel_crs != polygon_crs:
    #polygon = polygon.to_crs(epsg=sentinel_crs)
    polygon_buffered = polygon_buffered.to_crs(epsg=sentinel_crs)
    
for i in sds:
        with rio.open(sd[i]) as subdata:

            #clipped_image, clipped_transform = mask(subdata, polygon['geometry'], crop=True)
            clipped_image, clipped_transform = mask(subdata, polygon_buffered, crop=True)
            
            clipped_meta = subdata.meta.copy()
            clipped_meta.update({
                "height": clipped_image.shape[1],
                "width": clipped_image.shape[2],
                "transform": clipped_transform
            })

            pixel_size = subdata.res
            pixel_size = int(round(pixel_size[0]))
    
        print(clipped_image.shape)

        out_meta = clipped_meta.copy()
        out_meta.update({
            'driver':'GTiff'
        })

        with rio.open(f'{image_name}_{pixel_size}m_bands_sentinel_clipped.tif', 'w', **out_meta) as dst:
            dst.write(clipped_image)
            print(f'Saved: {image_name}_{pixel_size}m_bands_sentinel_clipped.tif')

(6, 1120, 2512)
Saved: S2B_MSIL2A_20170519T101019_N0500_R022_T33UVT_20231115T232025_10m_bands_sentinel_clipped.tif
(11, 560, 1256)
Saved: S2B_MSIL2A_20170519T101019_N0500_R022_T33UVT_20231115T232025_20m_bands_sentinel_clipped.tif
